In [27]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
#!pip install deepxde==0.10.0
import deepxde as dde
import scipy as sp
import scipy.interpolate as interp
from deepxde.backend import tf
from matplotlib import pyplot as plt
from numpy.core.shape_base import hstack
from scipy import signal
# from param import *
loc_drive=""
path_subrotinas="/home/taniel/Desktop/Projetos_Deep/BCS/subrotinas.py"
exec(compile(open('param_LEA.py', "rb").read(), 'param_LEA.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
exec(compile(open(path_subrotinas, "rb").read(), path_subrotinas, 'exec'))
exec(compile(open('exp_LEA.py', "rb").read(), 'exp_LEA.py', 'exec')) 

0.24108276567583545 7.559321788388188 [-11.492505101438962, 204.43014465827954]
Dados carregados


In [28]:
# =========================================================================
#  Define as entradas do BCS LEA   
# =========================================================================
#   freq = Cexp.referencia_frequencia_inversor(1:nsim)*0.1;                   % [Hz] frequencia de operacao
intervalo=np.array([int(1*3600),int(5*3600)])
#intervalo=np.array([int(0*3600),int(7.8*3600)])

#intervalo=np.array([1.8,3])
LEA=dados_LEA_Exp(loc_drive+'Dados_BCSLEA_20210818.mat',intervalo)
pm=LEA['pressao_manifold_coriolis']*1e5
pr=LEA['pressao_reservatorio']*1e5
pm_0=pm[0]
pr_0=pr[0]
u_0=np.array([LEA['referencia_frequencia_inversor'][0],LEA['valvula_pneumatica_topo'][0],pm_0,pr_0])
x_0=np.array([LEA['pressao_fundo'][0]*1e5,LEA['pressao_choke'][0]*1e5,LEA['vazao'][0]/3600])
u_0
nsim=LEA['tempo']+1
ts=LEA['Ts']
tempo_hora = np.arange(0,nsim*ts,ts)/3600

fk=LEA['referencia_frequencia_inversor']
zc=LEA['valvula_pneumatica_topo']
fk=np.reshape(fk, (nsim,1))
zc=np.reshape(zc, (nsim,1))
pm=np.reshape(pm, (nsim,1))
pr=np.reshape(pr, (nsim,1))
# time points
maxtime = nsim
time = np.linspace(0, nsim, nsim)
time.shape
nsim

1


144

In [29]:
def constante(valor,time_vector):
    return np.ones_like(time_vector)*valor

def ex_func(t):
  # loads exogenous inputs and interpolates intersample if necessary
    spline_f = sp.interpolate.Rbf(
        time, fk, function="thin_plate", smooth=0, episilon=0
    )
    spline_zc = sp.interpolate.Rbf(
        time, zc, function="thin_plate", smooth=0, episilon=0
    )
    spline_pman = sp.interpolate.Rbf(
        time, pm, function="thin_plate", smooth=0, episilon=0
    )
    spline_pres = sp.interpolate.Rbf(
        time, pr, function="thin_plate", smooth=0, episilon=0
    )
    #out=np.array([spline_f(t), spline_zc(t), spline_pman(t), spline_pres(t)])
    return [spline_f(t), spline_zc(t), spline_pman(t), spline_pres(t)]
    # out=np.hstack([np.array([spline_f(t)]).T,np.array([spline_zc(t)]).T,np.array([spline_pman(t)]).T,np.array([spline_pres(t)]).T ])
    # return out #np.transpose(out)


In [30]:
# Entradas
# Valores máximos e mínimos para normalização
#Entradas - conforme binder e pavlov
#========================================
pbc=Lim_c(pbhlim)
pwc=Lim_c(pwhlim)
qc=Lim_c(qlim)
pbmin=pbhlim[0]
pwmin=pwhlim[0]
qmin=qlim[0]


#rho=tf.log(tf.Variable(tf.exp(8.18)))*100 #836.8898 log(exp) impede valores negativos
rho=tf.exp(tf.Variable(6.7))
#rho=tf.Variable(tf.exp(818.0))
PI = tf.Variable(2.5)*1e-8 ##PI = 2.7e-8; # Well productivy index [m3/s/Pa]
xc=np.array([pbc,pwc,qc])
x0=np.array([pbmin,pwmin,qmin])
x_0=np.array([LEA['pressao_fundo'][0]*1e5,LEA['pressao_choke'][0]*1e5,LEA['vazao'][0]/3600])
x1=(LEA['pressao_fundo']*1e5-x0[0])/xc[0]
x2=(LEA['pressao_choke']*1e5-x0[1])/xc[1]
x3=(LEA['vazao']/3600-x0[2])/xc[2]
x1=x1.reshape(nsim,1)
x2=x2.reshape(nsim,1)
x3=x3.reshape(nsim,1)
tempo=np.arange(0,nsim)
tempo=tempo.reshape(len(x1),1)


# Fig_LEA_data(LEA)

# a=np.array([1,2,3,4,5,6,7])
# a[:3]
xss=x_0
xss
math.log(3.57e3)

8.180320874773685

In [31]:
F1c=2.92634e-05
F2c=0.000738599
Hc=215.9226497597185
qcc=0.0020328441729756536
F1lim=(0.000439365,0.000439365)
F2lim=(0.0110894,0.0110894)

In [32]:

def ED_BCS(t,x,ex_func):
    ## Montado o sistema de equa��es
    # Tensores (Estados)
    pbh = x[:,0:1]
    pwh = x[:,1:2]
    q = x[:,2:] #Vazão

    # fq=ex_func[:,0:1]
    # zc=ex_func[:,1:2]
    # pman=ex_func[:,2:3];#pman=0.12e5
    # pres=ex_func[:,3:4]

    # fq=ex_func[:,0]
    # zc=ex_func[:,1]
    # pman=ex_func[:,2];#pman=0.12e5
    # pres=ex_func[:,3]

    fq=ex_func[0]
    zc=ex_func[1]
    pman=ex_func[2];#pman=0.12e5
    pres=ex_func[3]

    # fq=50
    # zc=35
    # pman=0.12e5
    # pres=1.8e5

    #=============================================
    # Computing HEAD and pump pressure gain of LEA
    q0 = (qc*q+qmin) / Cq * (f0 / fq)
    H0 =  Head[0]*q0**4 +  Head[1]*q0**3 +  Head[2]*q0**2 + Head[3]*q0 + Head[4];
    H = CH * H0 * (fq / f0) ** 2  # Head
    #Pp = rho * g * H  # Dp
    #==============================================
    # Electrical power and electrical current computing
    #P0 = -2.3599e9 * q0 ** 3 - 1.8082e7 * q0 ** 2 + 4.3346e6 * q0 + 9.4355e4
    #P = Cp * P0 * (fq / f0) ** 3;  
    #I = Inp * P / Pnp  
    #==============================================
    # Computing two volumes frictions in LEA piping
    qan=q*qc+qmin # non normalized flow
    Re =(4*rho*qan)/(0.219*pi*mu); # Assuming volumes density are identicals
    fric=64/Re 
     
    F1 = (fric*qan**2*rho)/(2*pi*r1**3) #Frictional pressure drop above ESP (Assuming laminar flow)
    F2 = (fric*qan**2*rho)/(2*pi*r2**3) #Frictional pressure drop above ESP (Assuming laminar flow)
    #===========================================
    #===========================================
    # Computing intake pressure
    #pin = pbh*pbc+pbmin - rho * g * h1 - F1;
    # Computing Reservoir flow
    qr = PI * (pres - (pbh*pbc+pbmin));
    # Computing flow across Choke valvule
    qch = Cc * (zc) * tf.sqrt(tf.abs(pwh*pwc+pwmin - pman)); # Algumas operações precisam usar o prefixo do tensorflow 
    #============================================

    #Normalizing nonlinear terms
    ##########################
    qch=(qch-qch_lim[0])/qcc
    F1=(F1-F1lim[0])/F1c
    F2=(F2-F2lim[0])/F2c
    H=(H-H_lim[0])/Hc
    ###########################
    #xss=np.float32(np.array([2.0197e5,4.9338e5,4.2961e-4]));
  
    # SEDO

    dpbhdt = dde.grad.jacobian(x, t, i=0)
    dpwhdt = dde.grad.jacobian(x, t, i=1)
    dqdt = dde.grad.jacobian(x, t, i=2)

    return [
        dpbhdt - (1/pbc)*b1/V1*(qr - (q*qc+qmin)),
        dpwhdt - (1/pwc)*b2/V2*((q*qc+qmin) - (qcc*qch+qch_lim[0])),
        dqdt - (1/(qc*M))*(pbh*pbc+pbmin - (pwh*pwc+pwmin) - rho*g*hw - (F1c*F1+F1lim[0]) - (F2c*F2+F2lim[0]) + rho * g * (H*Hc+H_lim[0]))
        ]



In [33]:
def boundary(_, on_initial):
    return on_initial


x_0 = np.float32(xss)
xssn = (x_0-x0)/xc
geom = dde.geometry.TimeDomain(0, nsim)

ic1 = dde.IC(geom, lambda v: xssn[0], boundary, component=0)
ic2 = dde.IC(geom, lambda v: xssn[1], boundary, component=1)
ic3 = dde.IC(geom, lambda v: xssn[2], boundary, component=2)



observe_x1 = dde.PointSetBC(tempo, x1, component=0)
observe_x2 = dde.PointSetBC(tempo, x2, component=1)
observe_x3 = dde.PointSetBC(tempo, x3, component=2)

data = dde.data.PDE(
    #geom, ED_BCS, [ic1,ic2,ic3],
    #geom, ED_BCS, [ic1,ic2,ic3,observe_x1,observe_x2,observe_x3],
    #geom, ED_BCS, [ic1,ic2,ic3,observe_x1,observe_x2],
    geom, ED_BCS, [observe_x1,observe_x2],
    #geom, ED_BCS, [ic1,ic2,ic3,observe_x1],
    #solution=func,
    num_test=2000,
    num_domain=6000,
    #num_boundary=2,
    #anchors=tempo,
    auxiliary_var_function=ex_func
    )

In [34]:

passos=10000

layer_size = [1] + [30]*6 + [3]
activation = "tanh" # 
#activation = "sigmoid"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)
#                   batch_normalization="before")
# callbacks for storing results
fnamevar = "LEA_rho_PI.dat"
variable = dde.callbacks.VariableValue([rho,PI], period=100, filename=fnamevar)
model = dde.Model(data, net)
checker = dde.callbacks.ModelCheckpoint(
        "model_norm/modelo.ckpt", save_better_only=True)
early_stopping = dde.callbacks.EarlyStopping(min_delta=1e-4, patience=2000)
cb=[variable,checker,early_stopping]
#cb=[checker]
#cb=None

In [39]:
model.compile("adam", lr=0.001)

Compiling model...
'compile' took 1.043390 s



In [38]:
path_models='/home/taniel/Desktop/Projetos_Deep/BCS/Simulador_BCS_LEA/model_norm/'
model.restore(path_models+'modelo.ckpt-43000',verbose=1)

Restoring model from /home/taniel/Desktop/Projetos_Deep/BCS/Simulador_BCS_LEA/model_norm/modelo.ckpt-43000 ...

INFO:tensorflow:Restoring parameters from /home/taniel/Desktop/Projetos_Deep/BCS/Simulador_BCS_LEA/model_norm/modelo.ckpt-43000


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Variable_2/Adam not found in checkpoint
	 [[node save_1/RestoreV2 (defined at /home/taniel/anaconda3/envs/BCS/lib/python3.9/site-packages/deepxde/model.py:206) ]]

Original stack trace for 'save_1/RestoreV2':
  File "/home/taniel/anaconda3/envs/BCS/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/taniel/anaconda3/envs/BCS/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/taniel/anaconda3/envs/BCS/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/taniel/anaconda3/envs/BCS/lib/python3.9/site-packages/traitlets/config/application.py", line 845, in launch_instance
    

In [ ]:
plot_rho_PI('LEA_rho_PI.dat',43000,str_model='LEA')
modelo1='model_norm/modelo.ckpt-43000'
previsao(modelo1,[x1,x2,x3],tempo)

NameError: name 'plot_rho_PI' is not defined